In [1]:
import time

import requests

import soopProcess
import soopPull


In [2]:
bozeMag = soopProcess.processor(
    baseUrl='http://bozemanmagazine.com/events',
    linkRegex='/' + time.strftime('%Y/%m/%d') + '.*',
    urlType='crawled',
    linkBase='http://bozemanmagazine.com/events',
    today=True,
)


In [7]:
bozeMag.check_links()[0]

'http://bozemanmagazine.com/events/2018/06/15/62057_the_max-46089-1'

In [2]:
links = bozeMag.check_links()

In [3]:
links[0]

'http://bozemanmagazine.com/events//2018/06/15/61859_gvlt_discovery_walk_amphibious_adventures_kids'

In [5]:
msuData = msu.process(
    whenTag='dd',
    locationTag='dd',
    #locationRegex='^(?!.*(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday))',
    descTag='main',
    descClass='col-sm-8',
    important_weight=100,
    num_sentences=2
)

bozeMagData = bozeMag.process(
    whenTag='p',
    whenClass='date',
    descTag='div',
    descClass='description',
    # locationTag='address',
    # locationClass='location_text',
    important_weight=100,
    num_sentences=2
)


In [6]:
bozeMagData

,lxml,outUrl,text,food,title,day,when,location,details
6,<!DOCTYPE html> <!--[if lt IE 7]> <html class=...,http://bozemanmagazine.com/events//2018/06/15/...,Wild Sheep Foundation is partnering with South...,"[free, bbq]","Gear, Beer, and BBQ at Wild Sheep Foundation -...",6/15/2018,Fri. Jun. 15 3-7pm...,None,Wild Sheep Foundation is partnering with South...
13,<!DOCTYPE html> <!--[if lt IE 7]> <html class=...,http://bozemanmagazine.com/events//2018/06/15/...,"Read, sing, talk, play, write together! Presch...",[free],Preschool Storytime - Belgrade Community Library,6/15/2018,"Every Friday until Jul. 27, 11:15am-noon...",None,"Preschool storytime includes stories, songs, a..."
15,<!DOCTYPE html> <!--[if lt IE 7]> <html class=...,http://bozemanmagazine.com/events//2018/06/15/...,An evening of music performed by local musicia...,[coffee],Open Mic Night - Wild Joe's Coffee Spot,6/15/2018,Sat. Jun. 2 6pm...,None,"Bring your guitar, sitar, zither, poetry, come..."


In [4]:
soup = msuData.lxml[4]
import re

In [5]:
locationList = [x.text for x in soup.find_all('dd')]

In [6]:
locationList

['Saturday, May 12, 2018 at 10:00am          ',
 "            Leardini's, 50 Whitetail Lane, Ulm, MT",
 "\nFind all the information here.\n\nClinicians Perry and Courtney Frank from Standing on the Rock Boer Goats out of Missoula will answer all of your goat questions from feeding, marketing, fitting, showing, and other topics.\xa0 4-H members are encrouaged to bring their goats, fitting stands, and clippers to the boot camp.\xa0 If you just have a goat, that's ok!\xa0 Bring your goat and yourself along.\xa0 A goat lunch will be provided at noon.\nThe fee is $5/per member.\xa0 The fee is non-refundable and for lunch.\xa0 PLease make checks payable to Cascade County 4-H Livestock Leaders.\xa0\n\n\n\n",
 ' \r\n                                \r\n                Jerrica Seilstad                406-454-6980\njerrica.seilstad@montana.edu \n',
 '\n\n\n Google Calendar \n iCal\n\n\n\n']

In [10]:
r = re.compile('.*')

In [13]:
locationList = list(filter(r.search, locationList))
locationList[0].strip() + '...'

'Saturday, May 12, 2018 at 10:00am...'